In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [2]:
from bs4 import BeautifulSoup
import requests
from requests import get
import time
import random
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os

In [3]:

# Шаблон URL для парсинга
URL_TEMPLATE = 'https://www.avito.ru/all/avtomobili?p='
PAUSE_DURATION_SECONDS = 4
MAX_PAGES = 100  # Количество страниц для парсинга

def parse_page(soup):
    # Используем частичное совпадение для поиска блока объявления
    car_blocks = soup.find_all('div', class_=lambda x: x and 'iva-item-root' in x)

    # Инициализация списков для хранения данных
    names, prices, photos, descriptions, cities = [], [], [], [], []
    ad_ids, ad_descriptions, time_updates, parsing_times, ad_paths = [], [], [], [], []

    for car in car_blocks:
        # Текущее время парсинга
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        parsing_times.append(current_time)
        
        # Парсинг названия
        try:
            name_tag = car.find('a', class_=lambda x: x and 'styles-module-root' in x)
            name = name_tag['title'].strip() if name_tag and 'title' in name_tag.attrs else np.nan
        except:
            name = np.nan

        # Парсинг цены
        try:
            price_tag = car.find('meta', {"itemprop": "price"})
            price = price_tag['content'] if price_tag and 'content' in price_tag.attrs else np.nan
        except:
            price = np.nan

        # Парсинг фото
        try:
            photo_tag = car.find('img', class_=lambda x: x and 'photo-slider-image' in x)
            photo = photo_tag['src'] if photo_tag and 'src' in photo_tag.attrs else np.nan
        except:
            photo = np.nan
       
        # Парсинг описания
        try:
            description_tag = car.find('p', {"data-marker": "item-specific-params"})
            description = description_tag.text.strip() if description_tag else np.nan
        except:
            description = np.nan

        # Парсинг города и автодилера
        try:
            city_tags = car.find_all('span', class_=lambda x: x and 'styles-module-noAccent' in x)
            city = ', '.join([tag.text.strip() for tag in city_tags]) if city_tags else np.nan
        except:
            city = np.nan

        # Парсинг ad_id
        try:
            ad_id = car['data-item-id']  # Извлекаем ad_id из атрибута data-item-id
        except:
            ad_id = np.nan

        # Парсинг ad_description
        try:
            ad_description_tag = car.find('meta', {"itemprop": "description"})
            ad_description = ad_description_tag['content'].strip().replace('\n', '') if ad_description_tag and 'content' in ad_description_tag.attrs else np.nan
        except:
            ad_description = np.nan

        # Парсинг ссылки объявления
        try:
            ad_path_tag = car.find('a', class_=lambda x: x and 'styles-module-root' in x)
            ad_path = 'https://www.avito.ru' + ad_path_tag['href'].strip() if ad_path_tag and 'href' in ad_path_tag.attrs else np.nan
        except:
            ad_path = np.nan

        # Парсинг времени поднятия
        try:
            time_update_tag = car.find('p', class_=lambda x: x and 'styles-module-root' in x)
            time_update_full_text = time_update_tag.text.strip() if time_update_tag else np.nan
            time_update_words = time_update_full_text.split()
            time_update = ' '.join(time_update_words[:2]) if len(time_update_words) >= 2 else time_update_full_text
        except:
            time_update = np.nan
                            
        # Добавляем данные в списки
        names.append(name)
        prices.append(price)
        photos.append(photo)
        descriptions.append(description)
        cities.append(city)
        ad_ids.append(ad_id)
        ad_descriptions.append(ad_description)
        time_updates.append(time_update)
        ad_paths.append(ad_path)

    # Возвращаем DataFrame из спарсенных данных
    df_page = pd.DataFrame({
        'name': names,
        'price': prices,
        'photo': photos,
        'description': descriptions,
        'city': cities,
        'ad_id': ad_ids,
        'ad_description': ad_descriptions,
        'time_update': time_updates,
        'parsing_time': parsing_times,
        'ad_path': ad_paths
    })

    return df_page


def main():
    all_cars_df = pd.DataFrame()

    # Проверяем, существует ли файл с уже спарсенными данными
    if os.path.exists('cars_data.csv'):
        all_cars_df = pd.read_csv('cars_data.csv')

    for page_num in range(1, MAX_PAGES + 1):
        print(f"Парсим страницу {page_num}...")

        try:
            url = URL_TEMPLATE + str(page_num)
            driver.get(url)
            sleep(PAUSE_DURATION_SECONDS)

            # Получаем HTML-код страницы
            page_source = driver.page_source

            # Парсим HTML с BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Парсим данные на текущей странице и добавляем их в DataFrame
            df_page = parse_page(soup)
            all_cars_df = pd.concat([all_cars_df, df_page], ignore_index=True)
            all_cars_df['ad_id'] = all_cars_df['ad_id'].astype('int64')

            # Сохраняем промежуточный результат после каждой страницы
            all_cars_df.to_csv('cars_data.csv', index=False, encoding='utf-8-sig')

        except TimeoutException:
            print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
            continue
        except Exception as e:
            print(f"Произошла ошибка на странице {page_num}: {e}")
            continue

    print("Парсинг завершен.")

    # Сохраняем итоговый DataFrame
    all_cars_df.to_csv('cars_data.csv', index=False, encoding='utf-8-sig')


if __name__ == '__main__':
    try:
        service = Service(executable_path=ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        main()
    except Exception as e:
        print(e)
    finally:
        driver.quit()


Парсим страницу 1...
Парсим страницу 2...
Парсим страницу 3...
Парсинг завершен.


In [ ]:
# преобразование видимости датафрейма
pd.set_option('display.max_colwidth', None)

pd.set_option('display.width', 200)

In [ ]:
# Теперь сбрасываем эти опции
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')


In [ ]:
df_cars = pd.read_csv('cars_data.csv')
df_cars.head()